## Start from next session

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
-- Welcome to Snowflake Notebooks!
-- Try out a SQL cell to generate some data.
SELECT 'FRIDAY' as SNOWDAY, 0.2 as CHANCE_OF_SNOW
UNION ALL
SELECT 'SATURDAY',0.5
UNION ALL 
SELECT 'SUNDAY', 0.9;

In [ ]:
# Then, we can use the python name to turn cell2 into a Pandas dataframe
my_df = cell2.to_pandas()

# Chart the data
st.subheader("Chance of SNOW ❄️")
st.line_chart(my_df, x='SNOWDAY', y='CHANCE_OF_SNOW')

# Give it a go!
st.subheader("Try it out yourself and show off your skills 🥇")

# Experiment Set Up

## Context and Purpose of the Experiment

This worksheet is designed to validate the behavior of privileges in Snowflake, specifically addressing whether granting privileges on a schema automatically propagates to the objects (e.g., tables and views) contained within it. The experiment stems from the need to confirm the following hypothesis:

> "Granting privileges on a schema does not inherently grant the same privileges on the objects contained within that schema."

---

## Experiment Setup

- **Database and Schema**: A database named `playground` and a schema named `PUBLIC` were created to serve as the container for the experiment.
- **Table Creation**: A table named `ORDERS` was added to the `PUBLIC` schema to test access to objects within the schema.
- **Role Setup**: A role, `ANALYST_ROLE`, was created and assigned to a user. This role was granted `USAGE` privileges on both the database and schema.
- **Access Test**: After granting `USAGE` privileges, the ability of the role to query the table without explicitly granting privileges on the table was tested.
- **Explicit Privilege Grant**: To confirm the hypothesis, the `SELECT` privilege was explicitly granted on the `ORDERS` table, and the role's access was retested.

---

## Goals

1. Validate whether `USAGE` privileges on a schema extend to objects within the schema.
2. Demonstrate the necessity of explicit privilege grants for accessing schema-contained objects.
3. Clarify privilege hierarchy and ensure alignment with Snowflake's access control model.

---

## Conducting the Experiment

### **Setup Phase**
- Created the database, schema, table, and role.
- Granted the role `USAGE` privileges on the database and schema.

### **Testing Phase**
- Verified whether the role could query the table after receiving `USAGE` privileges.

### **Validation Phase**
- Explicitly granted the `SELECT` privilege on the table.
- Retested the role's ability to query the table.

---

## Results

Granting Schema Access Does grant Access to Tables

## Run Experiment

In [ ]:
SET my_user = (SELECT CURRENT_USER());
SELECT $my_user;

DROP TABLE IF EXISTS playground.PUBLIC.ORDERS;
CREATE TABLE playground.PUBLIC.ORDERS (
    ORDER_ID INT,
    ORDER_DATE DATE,
    CUSTOMER_ID INT,
    AMOUNT DECIMAL(10, 2)
);

DROP ROLE IF EXISTS ANALYST_ROLE;
CREATE ROLE ANALYST_ROLE;

GRANT USAGE ON SCHEMA playground.PUBLIC TO ROLE ANALYST_ROLE;

GRANT ROLE ANALYST_ROLE TO USER IDENTIFIER($my_user);

Now switch to `ANALYST_ROLE` and open a worksheet to execute the following:

```
INSERT INTO playground.PUBLIC.ORDERS (ORDER_ID, ORDER_DATE, CUSTOMER_ID, AMOUNT)
VALUES (999, '2024-01-01', 123, 100.50);
```
This should show as successfully executed


In [ ]:
-- Verify that we only granted schema access to ANALYST_ROLE
SHOW GRANTS TO ROLE ANALYST_ROLE;

Can we revoke the access so that ANALYST_ROLE is only given READ access to the public table?

In [ ]:
REVOKE USAGE ON SCHEMA playground.PUBLIC FROM ROLE ANALYST_ROLE;
SHOW GRANTS TO ROLE ANALYST_ROLE;

In [ ]:
GRANT SELECT ON playground.PUBLIC.ORDERS TO ROLE ANALYST_ROLE;
SHOW GRANTS TO ROLE ANALYST_ROLE;

Now let's try modifying the table again by switching to ANALYST_ROLE and running the following commands:

```
INSERT INTO playground.PUBLIC.ORDERS (ORDER_ID, ORDER_DATE, CUSTOMER_ID, AMOUNT)
VALUES (999, '2024-01-01', 123, 100.50);
```

In [ ]:
select * from playground.public.orders